<a href="https://colab.research.google.com/github/isabellasims/Google-colabs/blob/master/geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install geopandas
import geopandas as gpd #rename as gpd
import math

In [ ]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

Several arguments customize the appearance of the map:

location sets the initial center of the map. We use the latitude (42.32° N) and longitude (-71.0589° E) of the city of Boston.
tiles changes the styling of the map; in this case, we choose the OpenStreetMap style. If you're curious, you can find the other options listed here.
zoom_start sets the initial level of zoom of the map, where higher values zoom in closer to the map.
Take the time now to explore by zooming in and out, or by dragging the map in different directions.

In [ ]:
# Create a map

# location = lat long dc
# tiles changes the styling of the map
m_1 = folium.Map(location=[38.9072,-77.0369], tiles='openstreetmap', zoom_start=10)


# Display the map
m_1

# https://www.kaggle.com/alexisbcook/interactive-maps

In [ ]:
# Load the data
# data from http://crimemap.dc.gov/Download.aspx
crimes = pd.read_csv("/content/SearchResults (2).txt", encoding='latin-1')

# Drop rows with missing locations
#crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE.isin([
    'ROBBERY','ASSAULT W/DANGEROUS WEAPON', 'HOMICIDE','SEX ABUSE'])]
#crimes = crimes[crimes.REPORT_DATE>=2018]
# address = crimes.BLOCK[]
# address.head()

# Print the first five rows of the table
crimes.head()

,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,ANC,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,CCN,XBLOCK,YBLOCK,START_DATE,END_DATE
3,6/30/2020 1:03:01 AM,MIDNIGHT,ROBBERY,GUN,2922 - 2999 BLOCK OF 2ND STREET SE,7.0,707.0,8,8C,Cluster 39,010400 2,10400.0,Precinct 123,20094911,399591.0,130876.0,6/29/2020 11:17:32 PM,6/30/2020 1:02:57 AM
4,6/30/2020 1:10:06 AM,MIDNIGHT,ROBBERY,OTHERS,200 - 289 BLOCK OF TAYLOR STREET NE,4.0,405.0,5,5A,Cluster 19,009501 3,9501.0,Precinct 44,20094912,399680.0,141427.0,6/29/2020 11:47:39 PM,NaN
9,6/30/2020 3:13:25 AM,MIDNIGHT,ASSAULT W/DANGEROUS WEAPON,GUN,4100 - 4199 BLOCK OF BENNING ROAD NE,6.0,603.0,7,7F,Cluster 32,009603 1,9603.0,Precinct 102,20094943,404896.0,136155.0,6/30/2020 1:39:14 AM,6/30/2020 2:36:52 AM
10,7/1/2020 1:05:07 AM,MIDNIGHT,ROBBERY,OTHERS,3500 - 3899 BLOCK OF JAY STREET NE,6.0,601.0,7,7D,Cluster 30,009602 2,9602.0,Precinct 100,20094953,404579.0,137347.0,6/30/2020 2:00:15 AM,6/30/2020 3:30:18 AM
15,6/30/2020 7:35:56 AM,DAY,ROBBERY,GUN,3500 - 3703 BLOCK OF WHEELER ROAD SE,7.0,705.0,8,8C,Cluster 39,009804 1,9804.0,Precinct 122,20094979,400557.0,130171.0,6/30/2020 5:18:24 AM,6/30/2020 7:10:02 AM


In [40]:
homicides = crimes[crimes.OFFENSE == 'HOMICIDE']
#homicides

#homicides['BLOCK'] = homicides['BLOCK'].replace
blocks = homicides['BLOCK'][94].split()

#blocks
for i in homicides['BLOCK']:
  i = i.split()
  i = i[-3:]
  i = " ".join([str(elem) for elem in i])
  i = i + ", WASHINGTON DC "
 
 for 






94                   3500 - 3599 BLOCK OF STANTON ROAD SE
378                    5000 - 5099 BLOCK OF BASS PLACE SE
379                    5000 - 5099 BLOCK OF BASS PLACE SE
380                    5000 - 5099 BLOCK OF BASS PLACE SE
390                      1800 - 1899 BLOCK OF Q STREET SE
391                  4000 - 4199 BLOCK OF GRANT STREET NE
471                  4100  - 4199 BLOCK OF 14TH STREET NW
644                    1100 - 1199 BLOCK OF 46TH PLACE SE
646                   1400 - 1517 BLOCK OF 25TH STREET SE
677                  1700 - 1799 BLOCK OF BENNING ROAD NE
834                    5415 - 5599 BLOCK OF BASS PLACE SE
1152                  1926 - 1935 BLOCK OF 18TH STREET SE
1188                    700 - 716 BLOCK OF 21ST STREET NE
1192           4700  - 4931 BLOCK OF NORTH CAPITOL STREET
1195         1100 - 1199 BLOCK OF NEW HAMPSHIRE AVENUE NW
1204                    430 - 499 BLOCK OF 33RD STREET SE
1212                   1  - 199 BLOCK OF WALNUT STREET NW
1315          

In [ ]:
# Create a map
m_2 = folium.Map(location=[38.9072,-77.0369], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in crimes.iterrows():
    Marker([row['XBLOCK'], row['YBLOCK']]).add_to(m_2)

# Display the map
m_2

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "dlab.berkeley.edu-workshop")

location = geolocator.geocode('BENNING ROAD NE, Washington DC')
location

Location(Benning Road Northeast, Kingman Park, Washington, District of Columbia, 20002, United States of America, (38.8986652, -76.9755027, 0.0))